# 1D SSH Model with Variable Kappa Analysis

This notebook analyzes the 1dSSH model data with varying spectral localizer coupling (kappa) and disorder parameters.

**Outputs:**
- 3x2 grid plots comparing Hamiltonian and Spectral Localizer (DOS, eigenvalue index, IPR)
- r/z statistics vs disorder (multiple kappa values as lines)
- r/z statistics vs kappa (multiple disorder levels as lines)

In [ ]:
# Cell 1: Imports and Constants

import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
from pathlib import Path
from collections import defaultdict

# Plotting constants (matching analysis/3dAnderson-mobility-analysis.py)
COLORS = {'H': 'blue', 'SL': 'orange'}
FIGSIZE_2x3 = (18, 10)
FIGSIZE_2x2 = (18, 18)
TITLE_SIZE = 20
LABEL_SIZE = 20
SUPTITLE_SIZE = 24
ANNOTATION_PROPS = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Reference values for spectral statistics
GOE_R = 0.5295
POISSON_R = 0.386
GOE_Z = 0.5687
POISSON_Z = 0.5

print("Imports and constants loaded successfully.")

In [ ]:
# Cell 2: Utility Functions

def parse_1dSSHkappa_filename(filepath):
    """
    Parse a 1dSSHkappa data filename to extract parameters.
    
    Expected format: 1dSSHkappa_L{L}_w_{w}_{kappa}_disorder{start}-{end}_numEigs{n}_realizations{r}_{YYYY-MM-DD-HHMMSS}_*.dat
    
    Returns dict with keys: L, w, kappa, disorder_start, disorder_end, num_eigs, num_realizations, date, timestamp
    """
    basename = os.path.basename(filepath)
    params = {}
    
    # Extract L
    match = re.search(r'_L(\d+)_', basename)
    if match:
        params['L'] = int(match.group(1))
    
    # Extract w (hopping parameter)
    match = re.search(r'_w_([\d.]+)_', basename)
    if match:
        params['w'] = float(match.group(1))
    
    # Extract kappa (spectral localizer coupling)
    # Pattern: _w_{w}_{kappa}_disorder - kappa is after w and before disorder
    match = re.search(r'_w_[\d.]+_([\d.]+)_disorder', basename)
    if match:
        params['kappa'] = float(match.group(1))
    
    # Extract disorder range
    match = re.search(r'_disorder([\d.]+)-([\d.]+)_', basename)
    if match:
        params['disorder_start'] = float(match.group(1))
        params['disorder_end'] = float(match.group(2))
    
    # Extract number of eigenvalues
    match = re.search(r'_numEigs(\d+)_', basename)
    if match:
        params['num_eigs'] = int(match.group(1))
    
    # Extract number of realizations
    match = re.search(r'_realizations(\d+)_', basename)
    if match:
        params['num_realizations'] = int(match.group(1))
    
    # Extract date and timestamp (format: YYYY-MM-DD-HHMMSS)
    match = re.search(r'_(\d{4}-\d{2}-\d{2})-(\d{6})_', basename)
    if match:
        params['date'] = match.group(1)
        params['timestamp'] = match.group(2)
        params['datetime_str'] = f"{match.group(1)}-{match.group(2)}"
    
    return params


def find_1dSSHkappa_files(data_dir):
    """
    Find all 1dSSHkappa files and organize by parameters.
    
    For each kappa, keeps only the latest timestamp.
    Verifies all 6 required file types exist.
    
    Returns: Nested dict: {(L, w, d_start, d_end): {kappa: {'files': {...}, 'params': {...}}}}
    """
    # Required file suffixes
    required_suffixes = [
        '_H_eigval.dat',
        '_H_IPR.dat',
        '_spectral_localiser_eigval.dat',
        '_spectral_localiser_IPR.dat',
        '_seeds.dat',
        '_parameters.txt'
    ]
    
    # Find all parameter files first (they define complete file sets)
    param_files = glob.glob(os.path.join(data_dir, '1dSSHkappa_L*_parameters.txt'))
    
    # Group files by parameter set
    file_groups = defaultdict(lambda: defaultdict(dict))
    
    for param_file in param_files:
        params = parse_1dSSHkappa_filename(param_file)
        if not params:
            continue
            
        # Extract base path (everything before the suffix)
        base_path = param_file.replace('_parameters.txt', '')
        
        # Check all required files exist
        files_dict = {}
        all_exist = True
        for suffix in required_suffixes:
            file_path = base_path + suffix
            if os.path.exists(file_path):
                # Create a key from the suffix
                key = suffix.replace('.dat', '').replace('.txt', '').strip('_')
                files_dict[key] = file_path
            else:
                all_exist = False
                break
        
        if not all_exist:
            continue
        
        # Create parameter key
        L = params.get('L', 0)
        w = params.get('w', 0)
        d_start = params.get('disorder_start', 0)
        d_end = params.get('disorder_end', 0)
        kappa = params.get('kappa', 0)
        datetime_str = params.get('datetime_str', '')
        
        param_key = (L, w, d_start, d_end)
        
        # Keep only the latest timestamp for each kappa
        if kappa in file_groups[param_key]:
            existing_datetime = file_groups[param_key][kappa].get('datetime_str', '')
            if datetime_str > existing_datetime:
                file_groups[param_key][kappa] = {
                    'files': files_dict,
                    'params': params,
                    'datetime_str': datetime_str
                }
        else:
            file_groups[param_key][kappa] = {
                'files': files_dict,
                'params': params,
                'datetime_str': datetime_str
            }
    
    return dict(file_groups)


def load_parameters_file(parameters_txt_path):
    """
    Parse a parameters.txt file.
    
    Returns dict with disorder_resolution, L, w, kappa, rho, etc.
    """
    params = {}
    with open(parameters_txt_path, 'r') as f:
        for line in f:
            line = line.strip()
            if '=' in line:
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip()
                # Try to convert to appropriate type
                try:
                    if '.' in value:
                        params[key] = float(value)
                    else:
                        params[key] = int(value)
                except ValueError:
                    params[key] = value
    return params


def load_1dSSHkappa_data(file_dict, params_from_filename):
    """
    Load all data files using memmap for memory efficiency.
    
    Returns dict with loaded data + metadata.
    """
    files = file_dict['files']
    
    # Load parameters file for shape info
    params = load_parameters_file(files['parameters'])
    
    # Determine array shape
    disorder_resolution = int(params.get('disorder_resolution', 21))
    num_realizations = int(params.get('num_realizations', params_from_filename.get('num_realizations', 500)))
    L = int(params.get('L', params_from_filename.get('L', 3000)))
    
    shape = (disorder_resolution, num_realizations, L)
    
    # Create disorder values array
    disorder_start = float(params.get('disorder_start', params_from_filename.get('disorder_start', 0.0)))
    disorder_end = float(params.get('disorder_end', params_from_filename.get('disorder_end', 1.0)))
    disorder_values = np.linspace(disorder_start, disorder_end, disorder_resolution)
    
    # Load data using memmap
    data = {
        'params': params,
        'disorder_values': disorder_values,
        'shape': shape,
        'L': L,
        'disorder_resolution': disorder_resolution,
        'num_realizations': num_realizations,
        'kappa': params_from_filename.get('kappa', params.get('kappa', 0))
    }
    
    # Load eigenvalues and IPR using memmap
    if 'H_eigval' in files:
        data['H_eigval'] = np.memmap(files['H_eigval'], dtype='float64', mode='r', shape=shape)
    
    if 'H_IPR' in files:
        data['H_IPR'] = np.memmap(files['H_IPR'], dtype='float64', mode='r', shape=shape)
    
    if 'spectral_localiser_eigval' in files:
        data['SL_eigval'] = np.memmap(files['spectral_localiser_eigval'], dtype='float64', mode='r', shape=shape)
    
    if 'spectral_localiser_IPR' in files:
        data['SL_IPR'] = np.memmap(files['spectral_localiser_IPR'], dtype='float64', mode='r', shape=shape)
    
    return data


print("Utility functions defined successfully.")

In [ ]:
# Cell 3: Statistics Functions

def calculate_r(eigval):
    """
    Calculate the adjacent gap ratio r for a sorted array of eigenvalues.
    r = min(s_i, s_{i+1}) / max(s_i, s_{i+1})
    
    Returns mean r value.
    """
    eigval_sorted = np.sort(eigval)
    spacings = np.diff(eigval_sorted)
    
    min_vals = np.minimum(spacings[:-1], spacings[1:])
    max_vals = np.maximum(spacings[:-1], spacings[1:])
    
    r = np.divide(min_vals, max_vals, out=np.zeros_like(min_vals), where=max_vals != 0)
    return r.mean()


def calculate_z(eigval):
    """
    Calculate the next-nearest neighbor ratio z.
    
    Returns mean z value.
    """
    eigval_sorted = np.sort(eigval)
    s = np.diff(eigval_sorted)
    
    if len(s) < 5:
        return np.nan
    
    s_i_minus_2 = s[:-4]
    s_i_minus_1 = s[1:-3]
    s_i = s[2:-2]
    s_i_plus_1 = s[3:-1]
    
    nn = np.minimum(s_i, s_i_minus_1)
    n_other = np.maximum(s_i, s_i_minus_1)
    nnn_left = s_i_minus_1 + s_i_minus_2
    nnn_right = s_i + s_i_plus_1
    
    nnn = np.minimum.reduce([n_other, nnn_left, nnn_right])
    
    z = np.divide(nn, nnn, out=np.zeros_like(nn), where=nnn != 0)
    return z.mean()


def compute_rz_statistics(eigval_array):
    """
    Compute r and z statistics for each realization in an array.
    
    Args:
        eigval_array: 2D array of shape (num_realizations, num_eigenvalues)
        
    Returns:
        r_values: array of r values for each realization
        z_values: array of z values for each realization
    """
    num_realizations = eigval_array.shape[0]
    r_values = np.zeros(num_realizations)
    z_values = np.zeros(num_realizations)
    
    for i in range(num_realizations):
        r_values[i] = calculate_r(eigval_array[i, :])
        z_values[i] = calculate_z(eigval_array[i, :])
    
    return r_values, z_values


print("Statistics functions defined successfully.")

In [ ]:
# Cell 4: Plotting Functions

def plot_3x2_grid(H_eigval, H_IPR, SL_eigval, SL_IPR, title_suffix, save_path=None):
    """
    Create a 3x2 grid comparing Hamiltonian and Spectral Localizer.
    
    Layout:
        [H DOS]    [H Index Plot]    [H IPR]
        [SL DOS]   [SL Index Plot]   [SL IPR]
    """
    fig, axs = plt.subplots(2, 3, figsize=FIGSIZE_2x3, constrained_layout=True)
    
    # Column 0: DOS (horizontal histograms)
    axs[0, 0].hist(H_eigval, bins=100, density=True, orientation='horizontal',
                   color=COLORS['H'], alpha=0.8)
    axs[0, 0].set_title('Hamiltonian DOS', size=TITLE_SIZE)
    axs[0, 0].set_xlabel('P(E)', size=LABEL_SIZE)
    axs[0, 0].set_ylabel('Energy (E)', size=LABEL_SIZE)
    axs[0, 0].grid(True)
    axs[0, 0].set_axisbelow(True)
    
    axs[1, 0].hist(SL_eigval, bins=100, density=True, orientation='horizontal',
                   color=COLORS['SL'], alpha=0.8)
    axs[1, 0].set_title('Spectral Localiser DOS', size=TITLE_SIZE)
    axs[1, 0].set_xlabel('P(E)', size=LABEL_SIZE)
    axs[1, 0].set_ylabel('Eigenvalue', size=LABEL_SIZE)
    axs[1, 0].grid(True)
    axs[1, 0].set_axisbelow(True)
    
    # Column 1: Eigenvalues vs Index
    H_indices = np.arange(len(H_eigval))
    SL_indices = np.arange(len(SL_eigval))
    
    axs[0, 1].scatter(H_indices, np.sort(H_eigval), s=1, c=COLORS['H'], alpha=0.5)
    axs[0, 1].set_title('Hamiltonian Eigenvalues', size=TITLE_SIZE)
    axs[0, 1].set_xlabel('Index', size=LABEL_SIZE)
    axs[0, 1].set_ylabel('Energy (E)', size=LABEL_SIZE)
    axs[0, 1].grid(True)
    axs[0, 1].set_axisbelow(True)
    
    axs[1, 1].scatter(SL_indices, np.sort(SL_eigval), s=1, c=COLORS['SL'], alpha=0.5)
    axs[1, 1].set_title('Spectral Localiser Eigenvalues', size=TITLE_SIZE)
    axs[1, 1].set_xlabel('Index', size=LABEL_SIZE)
    axs[1, 1].set_ylabel('Eigenvalue', size=LABEL_SIZE)
    axs[1, 1].grid(True)
    axs[1, 1].set_axisbelow(True)
    
    # Column 2: IPR vs Index (sorted by eigenvalue)
    H_sort_idx = np.argsort(H_eigval)
    SL_sort_idx = np.argsort(SL_eigval)
    
    axs[0, 2].scatter(H_indices, H_IPR[H_sort_idx], s=1, c=COLORS['H'], alpha=0.5)
    axs[0, 2].set_title('Hamiltonian IPR', size=TITLE_SIZE)
    axs[0, 2].set_xlabel('Index', size=LABEL_SIZE)
    axs[0, 2].set_ylabel('IPR', size=LABEL_SIZE)
    axs[0, 2].grid(True)
    axs[0, 2].set_axisbelow(True)
    
    # Add average IPR annotation
    avg_H_IPR = np.mean(H_IPR)
    axs[0, 2].text(0.95, 0.95, f'avg={avg_H_IPR:.4f}', transform=axs[0, 2].transAxes,
                   fontsize=14, verticalalignment='top', horizontalalignment='right',
                   bbox=ANNOTATION_PROPS)
    
    axs[1, 2].scatter(SL_indices, SL_IPR[SL_sort_idx], s=1, c=COLORS['SL'], alpha=0.5)
    axs[1, 2].set_title('Spectral Localiser IPR', size=TITLE_SIZE)
    axs[1, 2].set_xlabel('Index', size=LABEL_SIZE)
    axs[1, 2].set_ylabel('IPR', size=LABEL_SIZE)
    axs[1, 2].grid(True)
    axs[1, 2].set_axisbelow(True)
    
    avg_SL_IPR = np.mean(SL_IPR)
    axs[1, 2].text(0.95, 0.95, f'avg={avg_SL_IPR:.4f}', transform=axs[1, 2].transAxes,
                   fontsize=14, verticalalignment='top', horizontalalignment='right',
                   bbox=ANNOTATION_PROPS)
    
    fig.suptitle(f'DOS, Eigenvalues and IPR: {title_suffix}', fontsize=SUPTITLE_SIZE)
    
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
    
    return fig, axs


def plot_rz_vs_disorder_for_kappas(data_by_kappa, save_path=None):
    """
    Create 2x2 grid with disorder on x-axis and multiple kappa values as lines.
    
    Layout:
        [H r vs disorder]    [H z vs disorder]
        [SL r vs disorder]   [SL z vs disorder]
    """
    fig, axs = plt.subplots(2, 2, figsize=FIGSIZE_2x2, constrained_layout=True)
    
    # Sort kappas for consistent legend ordering
    sorted_kappas = sorted(data_by_kappa.keys())
    
    # Color map for different kappas
    cmap = plt.cm.viridis
    colors = [cmap(i / len(sorted_kappas)) for i in range(len(sorted_kappas))]
    
    for kappa_idx, kappa in enumerate(sorted_kappas):
        data = data_by_kappa[kappa]
        disorder_values = data['disorder_values']
        num_disorder = len(disorder_values)
        num_realizations = data['num_realizations']
        sqrt_n = np.sqrt(num_realizations)
        
        # Compute statistics for each disorder level
        H_r_means, H_r_stds = [], []
        H_z_means, H_z_stds = [], []
        SL_r_means, SL_r_stds = [], []
        SL_z_means, SL_z_stds = [], []
        
        for d_idx in range(num_disorder):
            # Compute r and z for each realization
            H_r_vals, H_z_vals = compute_rz_statistics(data['H_eigval'][d_idx, :, :])
            SL_r_vals, SL_z_vals = compute_rz_statistics(data['SL_eigval'][d_idx, :, :])
            
            H_r_means.append(np.mean(H_r_vals))
            H_r_stds.append(np.std(H_r_vals) / sqrt_n)
            H_z_means.append(np.mean(H_z_vals))
            H_z_stds.append(np.std(H_z_vals) / sqrt_n)
            
            SL_r_means.append(np.mean(SL_r_vals))
            SL_r_stds.append(np.std(SL_r_vals) / sqrt_n)
            SL_z_means.append(np.mean(SL_z_vals))
            SL_z_stds.append(np.std(SL_z_vals) / sqrt_n)
        
        color = colors[kappa_idx]
        label = f'κ={kappa}'
        
        # Plot H statistics
        axs[0, 0].errorbar(disorder_values, H_r_means, yerr=H_r_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        axs[0, 1].errorbar(disorder_values, H_z_means, yerr=H_z_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        
        # Plot SL statistics
        axs[1, 0].errorbar(disorder_values, SL_r_means, yerr=SL_r_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        axs[1, 1].errorbar(disorder_values, SL_z_means, yerr=SL_z_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
    
    # Add reference lines for r plots (left column)
    for ax in [axs[0, 0], axs[1, 0]]:
        ax.axhline(y=GOE_R, color='red', linestyle='--', label='GOE', alpha=0.7)
        ax.axhline(y=POISSON_R, color='green', linestyle='--', label='Poisson', alpha=0.7)
        ax.set_xlabel('Disorder Strength', size=LABEL_SIZE)
        ax.set_ylabel('r', size=LABEL_SIZE)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True)
    
    # Add reference lines for z plots (right column)
    for ax in [axs[0, 1], axs[1, 1]]:
        ax.axhline(y=GOE_Z, color='red', linestyle='--', label='GOE', alpha=0.7)
        ax.axhline(y=POISSON_Z, color='green', linestyle='--', label='Poisson', alpha=0.7)
        ax.set_xlabel('Disorder Strength', size=LABEL_SIZE)
        ax.set_ylabel('z', size=LABEL_SIZE)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True)
    
    # Set titles
    axs[0, 0].set_title('r of Hamiltonian', size=TITLE_SIZE)
    axs[0, 1].set_title('z of Hamiltonian', size=TITLE_SIZE)
    axs[1, 0].set_title('r of Spectral Localizer', size=TITLE_SIZE)
    axs[1, 1].set_title('z of Spectral Localizer', size=TITLE_SIZE)
    
    fig.suptitle('Spectral Statistics vs Disorder (multiple κ values)', fontsize=SUPTITLE_SIZE)
    
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
    
    return fig, axs


def plot_rz_vs_kappa_for_disorders(data_by_kappa, selected_disorder_indices, save_path=None):
    """
    Create 2x2 grid with kappa on x-axis and multiple disorder levels as lines.
    
    Layout:
        [H r vs kappa]    [H z vs kappa]
        [SL r vs kappa]   [SL z vs kappa]
    """
    fig, axs = plt.subplots(2, 2, figsize=FIGSIZE_2x2, constrained_layout=True)
    
    # Get kappas and disorder values
    sorted_kappas = sorted(data_by_kappa.keys())
    first_data = data_by_kappa[sorted_kappas[0]]
    disorder_values = first_data['disorder_values']
    num_realizations = first_data['num_realizations']
    sqrt_n = np.sqrt(num_realizations)
    
    # Color map for different disorder levels
    cmap = plt.cm.plasma
    colors = [cmap(i / len(selected_disorder_indices)) for i in range(len(selected_disorder_indices))]
    
    for d_color_idx, d_idx in enumerate(selected_disorder_indices):
        W = disorder_values[d_idx]
        
        H_r_means, H_r_stds = [], []
        H_z_means, H_z_stds = [], []
        SL_r_means, SL_r_stds = [], []
        SL_z_means, SL_z_stds = [], []
        
        for kappa in sorted_kappas:
            data = data_by_kappa[kappa]
            
            # Compute r and z for this disorder level across all realizations
            H_r_vals, H_z_vals = compute_rz_statistics(data['H_eigval'][d_idx, :, :])
            SL_r_vals, SL_z_vals = compute_rz_statistics(data['SL_eigval'][d_idx, :, :])
            
            H_r_means.append(np.mean(H_r_vals))
            H_r_stds.append(np.std(H_r_vals) / sqrt_n)
            H_z_means.append(np.mean(H_z_vals))
            H_z_stds.append(np.std(H_z_vals) / sqrt_n)
            
            SL_r_means.append(np.mean(SL_r_vals))
            SL_r_stds.append(np.std(SL_r_vals) / sqrt_n)
            SL_z_means.append(np.mean(SL_z_vals))
            SL_z_stds.append(np.std(SL_z_vals) / sqrt_n)
        
        color = colors[d_color_idx]
        label = f'W={W:.2f}'
        
        # Plot H statistics
        axs[0, 0].errorbar(sorted_kappas, H_r_means, yerr=H_r_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        axs[0, 1].errorbar(sorted_kappas, H_z_means, yerr=H_z_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        
        # Plot SL statistics
        axs[1, 0].errorbar(sorted_kappas, SL_r_means, yerr=SL_r_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
        axs[1, 1].errorbar(sorted_kappas, SL_z_means, yerr=SL_z_stds,
                           label=label, marker='o', capsize=3, linestyle='-', color=color)
    
    # Add reference lines for r plots (left column)
    for ax in [axs[0, 0], axs[1, 0]]:
        ax.axhline(y=GOE_R, color='red', linestyle='--', label='GOE', alpha=0.7)
        ax.axhline(y=POISSON_R, color='green', linestyle='--', label='Poisson', alpha=0.7)
        ax.set_xlabel('κ (Spectral Localizer Coupling)', size=LABEL_SIZE)
        ax.set_ylabel('r', size=LABEL_SIZE)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True)
    
    # Add reference lines for z plots (right column)
    for ax in [axs[0, 1], axs[1, 1]]:
        ax.axhline(y=GOE_Z, color='red', linestyle='--', label='GOE', alpha=0.7)
        ax.axhline(y=POISSON_Z, color='green', linestyle='--', label='Poisson', alpha=0.7)
        ax.set_xlabel('κ (Spectral Localizer Coupling)', size=LABEL_SIZE)
        ax.set_ylabel('z', size=LABEL_SIZE)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True)
    
    # Set titles
    axs[0, 0].set_title('r of Hamiltonian', size=TITLE_SIZE)
    axs[0, 1].set_title('z of Hamiltonian', size=TITLE_SIZE)
    axs[1, 0].set_title('r of Spectral Localizer', size=TITLE_SIZE)
    axs[1, 1].set_title('z of Spectral Localizer', size=TITLE_SIZE)
    
    fig.suptitle('Spectral Statistics vs κ (multiple disorder levels)', fontsize=SUPTITLE_SIZE)
    
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
    
    return fig, axs


print("Plotting functions defined successfully.")

In [ ]:
# Cell 5: Configuration (User-Editable)

# =============================================================================
# DATA DIRECTORY
# =============================================================================
DATA_DIR = '../data/'

# =============================================================================
# FILE SELECTION
# =============================================================================
# Available kappa values: 0.5, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 2.0
selected_kappas = [0.5, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 2.0]  # Or a subset

# Target system parameters
target_L = 3000
target_w = 1.0
target_disorder_start = 0.0
target_disorder_end = 1.0

# =============================================================================
# PLOT OPTIONS
# =============================================================================
# For 3x2 grids: which disorder indices to plot?
# None = auto-select [0, mid, -1] (low, medium, high disorder)
# Or specify: [0, 5, 10, 20] for specific indices
disorder_indices_to_plot = None

# For r/z vs kappa: which disorder level indices to include?
# None = all disorder levels
disorder_indices_for_rz_vs_kappa = [0, 10, 20]  # Example: low, mid, high

# =============================================================================
# SAVE OPTIONS
# =============================================================================
SAVE_FIGURES = True
FIGURE_DIR = '../figures/'

print("Configuration loaded.")
print(f"  Data directory: {DATA_DIR}")
print(f"  Selected kappas: {selected_kappas}")
print(f"  Target L={target_L}, w={target_w}")
print(f"  Save figures: {SAVE_FIGURES}")

In [ ]:
# Cell 6: Analysis 1 - 3x2 Grid Visualization

print("Loading 1dSSHkappa files...")
all_files = find_1dSSHkappa_files(DATA_DIR)

if not all_files:
    print("WARNING: No 1dSSHkappa files found in the data directory!")
    print(f"Searched in: {DATA_DIR}")
else:
    print(f"Found {len(all_files)} parameter set(s).")
    for param_key, kappa_dict in all_files.items():
        L, w, d_start, d_end = param_key
        print(f"  L={L}, w={w}, disorder={d_start}-{d_end}: {len(kappa_dict)} kappa values")

# Filter by target parameters and create 3x2 grids
for param_key, kappa_dict in all_files.items():
    L, w, d_start, d_end = param_key
    
    # Check if this matches our target parameters
    if L != target_L or w != target_w:
        continue
    if d_start != target_disorder_start or d_end != target_disorder_end:
        continue
    
    print(f"\nProcessing parameter set: L={L}, w={w}, disorder={d_start}-{d_end}")
    
    for kappa, file_info in sorted(kappa_dict.items()):
        if kappa not in selected_kappas:
            continue
        
        print(f"\n  Loading kappa={kappa}...")
        data = load_1dSSHkappa_data(file_info, file_info['params'])
        
        # Determine disorder indices to plot
        num_disorder = data['disorder_resolution']
        if disorder_indices_to_plot is None:
            d_indices = [0, num_disorder // 2, num_disorder - 1]
        else:
            d_indices = disorder_indices_to_plot
        
        # Create 3x2 grid for each selected disorder level
        for d_idx in d_indices:
            if d_idx >= num_disorder:
                print(f"    Skipping disorder index {d_idx} (out of range)")
                continue
            
            W = data['disorder_values'][d_idx]
            print(f"    Creating 3x2 grid for W={W:.2f}...")
            
            # Flatten data across realizations
            H_eigval_flat = data['H_eigval'][d_idx, :, :].flatten()
            H_IPR_flat = data['H_IPR'][d_idx, :, :].flatten()
            SL_eigval_flat = data['SL_eigval'][d_idx, :, :].flatten()
            SL_IPR_flat = data['SL_IPR'][d_idx, :, :].flatten()
            
            # Plot
            title = f'κ={kappa}, W={W:.2f}, L={L}'
            save_name = os.path.join(FIGURE_DIR, f'1dSSHkappa_3x2_k{kappa}_W{W:.2f}.png') if SAVE_FIGURES else None
            
            fig, axs = plot_3x2_grid(H_eigval_flat, H_IPR_flat, SL_eigval_flat, SL_IPR_flat, title, save_name)
            plt.show()

print("\n3x2 grid visualization complete.")

In [ ]:
# Cell 7: Analysis 2 - r/z Statistics

print("Loading data for r/z statistics analysis...")

# =============================================================================
# Part A: Load and Organize Data
# =============================================================================

grouped_data = {}
for param_key, kappa_dict in all_files.items():
    L, w, d_start, d_end = param_key
    
    # Check target parameters
    if L != target_L or w != target_w:
        continue
    if d_start != target_disorder_start or d_end != target_disorder_end:
        continue
    
    kappa_group = {}
    for kappa, file_info in sorted(kappa_dict.items()):
        if kappa not in selected_kappas:
            continue
        
        print(f"  Loading kappa={kappa}...")
        data = load_1dSSHkappa_data(file_info, file_info['params'])
        kappa_group[kappa] = data
    
    if kappa_group:
        grouped_data[param_key] = kappa_group

if not grouped_data:
    print("WARNING: No data matched the target parameters!")
else:
    print(f"\nLoaded {sum(len(kg) for kg in grouped_data.values())} kappa datasets.")

# =============================================================================
# Part B: Plot r/z vs Disorder
# =============================================================================

print("\nGenerating r/z vs Disorder plots...")
for param_key, kappa_dict in grouped_data.items():
    L, w, d_start, d_end = param_key
    print(f"  Parameter set: L={L}, w={w}, disorder={d_start}-{d_end}")
    
    save_path = os.path.join(FIGURE_DIR, f'1dSSHkappa_rz_vs_disorder_L{L}.png') if SAVE_FIGURES else None
    fig, axs = plot_rz_vs_disorder_for_kappas(kappa_dict, save_path=save_path)
    plt.show()

# =============================================================================
# Part C: Plot r/z vs Kappa
# =============================================================================

print("\nGenerating r/z vs Kappa plots...")
for param_key, kappa_dict in grouped_data.items():
    L, w, d_start, d_end = param_key
    print(f"  Parameter set: L={L}, w={w}, disorder={d_start}-{d_end}")
    
    # Determine disorder indices
    first_data = kappa_dict[list(kappa_dict.keys())[0]]
    num_disorder = first_data['disorder_resolution']
    
    if disorder_indices_for_rz_vs_kappa is None:
        d_indices = list(range(num_disorder))
    else:
        d_indices = [idx for idx in disorder_indices_for_rz_vs_kappa if idx < num_disorder]
    
    print(f"    Using disorder indices: {d_indices}")
    print(f"    Corresponding W values: {[first_data['disorder_values'][i] for i in d_indices]}")
    
    save_path = os.path.join(FIGURE_DIR, f'1dSSHkappa_rz_vs_kappa_L{L}.png') if SAVE_FIGURES else None
    fig, axs = plot_rz_vs_kappa_for_disorders(kappa_dict, d_indices, save_path=save_path)
    plt.show()

print("\nr/z statistics analysis complete.")